## TODO:

- [x] import CORA dataset
- [ ] implement our approach: consider k-hop neighbors' input features and the current node's hidden state
- [ ] experiment a first model with GCN as the message passing scheme (later we'll experiment with GAT and GraphSAGE)
- [ ] track over-smoothing with **MAD** and **MADGap** over: 
    - [ ] K hops considered
    - [ ] epochs

#### Side notes

- Try to inverse the k-hop neighbors (start from remote and move down)
- Early stop for nodes that don't have k-hop neighbors (just pass the hidden state)

In [ ]:
# setup colab environment
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [ ]:
# import utility functions
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-linear-receptive-fields/main/utils.py
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-linear-receptive-fields/main/data.py

In [ ]:
from utils import mean_average_distance, mean_average_distance_gap
from data import load_dataset

OSError: ignored

In [ ]:
# import libraries
import torch
import torch_geometric as tg

In [ ]:
with open('data.py', 'r') as f:
    print(f.readline())

from torch_geometric.datasets import Planetoid, TUDataset, PPI, QM9, Amazon, Reddit2



In [ ]:
path = osp.join(os.getcwd(), 'data')
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0] # only graph of the dataset

MAD_cora = mean_average_distance(x=G.x)
MADGap_cora = mean_average_distance_gap(x=G.x, adj_matrix=tg.utils.to_dense_adj(G.edge_index)[0])
print(f'Initial MAD for Cora: {MAD_cora}')
print(f'Initial MADGap for Cora: {MADGap_cora}')

Processing...
Done!
Initial MAD for Cora: 1278.501953125
Initial MADGap for Cora: 1274.7158203125


# Our Approach

In [ ]:
class KHopNet(torch.nn.Module):
    def __init__(self, k):
        self(KHopNet, self).__init__()
    
    def forward(self, x):
        pass
    
    def fit(self, dataset, epochs, lr):
        optimizer = torch.optim.SGD(self.parameters(), lr=lr)
        criterion = torch.nn.CrossEntropy()
        
        for epoch in range(epochs):
            total_loss = 0
            # TODO: feed forward, backpropagate
            
            # forward
            # output = self(data)
            loss = criterion(output, label)
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss